In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Import of keras model and hidden layers for our convolutional network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten

print(tf.__version__)

2.4.1


Model trained in two categories fist, and palm. 
Trained on https://www.kaggle.com/gti-upm/leapgestrecog/version/1


In [4]:
batch_size = 64
img_size = (128,128,1)


_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255.,
    validation_split=0.3
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255.,
)


train_generator = _datagen.flow_from_directory(
    directory='../../../DATA/data/train/images',
    shuffle=True,
    target_size=(img_size[0],img_size[1]),
    subset="training",
    color_mode='grayscale',
    batch_size=batch_size)
print('Train generator created\n')

valid_generator = _datagen.flow_from_directory(
    directory='../../../DATA/data/train/images',
    shuffle=True,
    target_size=img_size,
    subset="validation",
    color_mode='grayscale',
    batch_size=batch_size)
print('Valid generator created\n')


test_generator = test_datagen.flow_from_directory(
    directory = '../../../DATA/data/test/images',
    shuffle = True,
    target_size = (img_size[0],img_size[1]),
    color_mode='grayscale',
    batch_size=batch_size*4

)
print('Train generator created\n')

Found 5040 images belonging to 2 classes.
Train generator created

Found 2160 images belonging to 2 classes.
Valid generator created

Found 799 images belonging to 2 classes.
Train generator created



In [3]:
# Construction of model
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=img_size) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss=tf.keras.losses.BinaryCrossentropy(), # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

model.summary()


SyntaxError: invalid syntax (<ipython-input-3-77891a198b72>, line 4)

In [10]:
steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

training = model.fit(train_generator,
                               epochs=5,
                               steps_per_epoch=steps_per_epoch,
                               validation_data=valid_generator,
                               validation_steps=validation_steps,
                               verbose=1
                               )


Epoch 1/5
78/78 [==============================] - 109s 1s/step - loss: 0.3478 - accuracy: 0.8138 - val_loss: 3.7948e-04 - val_accuracy: 1.0000
Epoch 2/5
78/78 [==============================] - 111s 1s/step - loss: 0.0049 - accuracy: 0.9989 - val_loss: 7.1825e-05 - val_accuracy: 1.0000
Epoch 3/5
78/78 [==============================] - 107s 1s/step - loss: 5.0550e-05 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/5
78/78 [==============================] - 128s 2s/step - loss: 7.7441e-05 - accuracy: 1.0000 - val_loss: 8.8576e-05 - val_accuracy: 1.0000
Epoch 5/5
78/78 [==============================] - 135s 2s/step - loss: 1.0676e-05 - accuracy: 1.0000 - val_loss: 9.9460e-05 - val_accuracy: 1.0000


In [21]:
model.evaluate(test_generator)

4/4 [==============================] - 5s 1s/step - loss: 2.4188e-05 - accuracy: 1.0000


[2.4188415409298614e-05, 1.0]

In [20]:
model.save('trained_palm_fist.h5')
